# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Subclases

In [27]:
query = '''
SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LAA.ARTC_ARTC_COD IN ('250111000', '240037000', '240036000', '141255000', '260849000', '330177000', '263986000')
    AND LAA.ARTC_ARTC_DESC NOT IN ('ACEITE DE OLIVA EXTRA VIRGEN LAMASIA 500.00 ML', 'GALLETITAS CLUB SOCIAL CLASICA 200.00 G')
    AND SUB.SUB_NAME NOT ILIKE '%NO%ESPECIFICADO%'
'''

cursor.execute(query)
subclases = cursor.fetch_pandas_all()
subclases.head(2)

SUB_NAME    SUBCLASE
0  JABONES DETERGENTE LIQUIDOS PARA ROPA  8800030003
1                            NO RELLENAS  1800010007

# Abandonadores

In [28]:
# antiguamente llamado Segmento 5
# Clientes que en el ultimo mes no vengan a tata y tengan Score 4 o 5 en la subclase

In [29]:
tuple(subclases['SUBCLASE'].astype(str))

('8800030003',
 '1800010007',
 '200010001',
 '200010002',
 '1800020002',
 '2900020002',
 '1100010002')

In [30]:
query = '''
WITH VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND LSS.SCORE >= 4
        AND LSS.SUBCLASE IN {subclases_snow}
        AND (CONT.CORREO IS NOT NULL OR CONT.CELULAR IS NOT NULL)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_5 = cursor.fetch_pandas_all()

segm_5 = segm_5.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_5.shape[0])
segm_5.head(2)

134472


SOCI_SOCI_ID    SUBCLASE               CORREO   CELULAR  \
0       1749445   200010002                 None  99860906   
1         99797  1800020002  maura10@hotmail.com  92035846   

              SUB_NAME  
0         AGUA SIN GAS  
1  ARTESANALES SALADAS

# Activos

In [31]:
# antiguamente llamado Segmento 3
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [32]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    ),

COMPRAS_SUBCLASES AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID || ' - ' || IM.SUBCLASE AS KEY
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND FFM.TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
    )


SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
        AND LSS.SCORE = 5
        AND LSS.SUBCLASE IN {subclases_snow}
        AND LSS.SOCI_SOCI_ID || ' - ' || LSS.SUBCLASE NOT IN (SELECT * FROM COMPRAS_SUBCLASES)
        AND (CONT.CORREO IS NOT NULL OR CONT.CELULAR IS NOT NULL)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_3 = cursor.fetch_pandas_all()

segm_3 = segm_3.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_3.shape[0])
segm_3.head(2)

153153


SOCI_SOCI_ID   SUBCLASE                      CORREO   CELULAR      SUB_NAME
0       1827969  200010002                        None  97154175  AGUA SIN GAS
1        401040  200010002  marialucia_pch@hotmail.com  99507284  AGUA SIN GAS

# Excels

In [33]:
for i in segm_3['SUB_NAME'].unique():
    print(i)
    df_aux = segm_3[segm_3['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata 8 Activos - {i}.xlsx', index = False)

AGUA SIN GAS
CANOLA
HAMBURGUESAS VACUNA INDIVIDUAL
JABONES DETERGENTE LIQUIDOS PARA ROPA
AGUA CON GAS
ARTESANALES SALADAS
NO RELLENAS


In [34]:
for i in segm_5['SUB_NAME'].unique():
    print(i)
    df_aux = segm_5[segm_5['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata 8 Abandonadores - {i}.xlsx', index = False)

AGUA SIN GAS
ARTESANALES SALADAS
AGUA CON GAS
NO RELLENAS
HAMBURGUESAS VACUNA INDIVIDUAL
CANOLA
JABONES DETERGENTE LIQUIDOS PARA ROPA


# SNOW

In [37]:
segm_5['PERFIL_CLIENTE'] = 'ABANDONADOR'
segm_3['PERFIL_CLIENTE'] = 'ACTIVO'
ambos = pd.concat([segm_5, segm_3])
ambos['OFERTATA'] = 'Ofertata 8'
ambos['FECHA'] = '2024-02-22'

In [40]:
success, nchunks, nrows, _ = write_pandas(ctx,
    ambos.drop(['SUB_NAME'], axis = 1), table_name='SEGMENTOS_OFERTATA', database= 'SANDBOX_PLUS', schema = 'DWH')
print('listo')

listo


# Litoral

In [35]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -24, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    ),

VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -6, CURRENT_DATE)
    )

    SELECT
        SOCI_SOCI_ID,
        CORREO,
        CELULAR
    FROM
        SANDBOX_PLUS.DWH.CONTACTABLES_2
    WHERE
        SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
        AND (CORREO IS NOT NULL OR CELULAR IS NOT NULL)

'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
litoral_abandonador_6_meses = cursor.fetch_pandas_all()

litoral_abandonador_6_meses = litoral_abandonador_6_meses[~litoral_abandonador_6_meses['SOCI_SOCI_ID'].isin(segm_3['SOCI_SOCI_ID'])]

litoral_abandonador_6_meses = litoral_abandonador_6_meses[~litoral_abandonador_6_meses['SOCI_SOCI_ID'].isin(segm_5['SOCI_SOCI_ID'])]

print(litoral_abandonador_6_meses.shape[0])
litoral_abandonador_6_meses.head(2)

31379


SOCI_SOCI_ID CORREO   CELULAR
0       1040556   None  95111670
1       1505577   None  99373681

In [36]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -24, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    ),

VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -12, CURRENT_DATE)
    )

    SELECT
        SOCI_SOCI_ID,
        CORREO,
        CELULAR
    FROM
        SANDBOX_PLUS.DWH.CONTACTABLES_2
    WHERE
        SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
        AND (CORREO IS NOT NULL OR CELULAR IS NOT NULL)

'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
litoral_abandonador_12_meses = cursor.fetch_pandas_all()

litoral_abandonador_12_meses = litoral_abandonador_12_meses[~litoral_abandonador_12_meses['SOCI_SOCI_ID'].isin(segm_3['SOCI_SOCI_ID'])]

litoral_abandonador_12_meses = litoral_abandonador_12_meses[~litoral_abandonador_12_meses['SOCI_SOCI_ID'].isin(segm_5['SOCI_SOCI_ID'])]

litoral_abandonador_12_meses = litoral_abandonador_12_meses[~litoral_abandonador_12_meses['SOCI_SOCI_ID'].isin(litoral_abandonador_6_meses['SOCI_SOCI_ID'])]

print(litoral_abandonador_12_meses.shape[0])
litoral_abandonador_12_meses.head(2)

0


Empty DataFrame
Columns: [SOCI_SOCI_ID, CORREO, CELULAR]
Index: []

# Fin